In [4]:
import tensorflow as tf
import tensorflow_probability as tfp
import math
import numpy as np
from numpy import random
from math import pi
import matplotlib.pyplot as plt
from math import sqrt
from  sklearn import mixture
import scipy.sparse
from scipy.stats import norm
from scipy.stats import multivariate_normal
tfd = tfp.distributions
from sklearn.utils import shuffle


In [5]:
# Unnormalized Data Distribution
def pm0(x, m, s):
    mvn = tfd.MultivariateNormalFullCovariance(
    loc=m,
    covariance_matrix=s)
    k = len(m)
    return np.sqrt(np.linalg.det(s)*(2*pi)**k)*mvn.prob(x);

# Noise Distribution
def pn(x, m0, s0):
    mvn = tfd.MultivariateNormalFullCovariance(
    loc=m0,
    covariance_matrix=s0)
    
    return mvn.prob(x);

In [6]:
## This class is a helper that  will be instantiated in objects containing results from A Gradient Descent

class Gradient:

    # instance attribute
    #Receive those attributes from
    def __init__(self, cte,mu=[],sigma=[], error_mu =[],error_sigma=[], error_cte=[], ctes=[],mus=[],sigmas=[]):
        self.cte = cte
        self.mu = mu
        self.sigma = sigma
        
        self.error_mu = error_mu
        self.error_sigma = error_sigma
        self.error_cte = error_cte 
        
        self.ctes = ctes
        self.mus = mus
        self.sigmas = sigmas
        

In [7]:
def obj_f2():  ##  Generator objective function
    
    g1= a*Z1 + b*Z2 + mu[0]
    g2 = c*Z1 +  d*Z2 + mu[1]

    g= tf.stack([g1,g2],1)


    p_n = pn( g,m0, S) # noise (generator) density 
    p_m = pm0(g, m_data, s_data) #data density
    
    
    temp = tf.math.reduce_mean( tf.math.log(p_n/(cte*p_m+p_n)))
   
    return tf.math.reduce_mean( tf.math.log(p_n/(cte*p_m+p_n)))

#### Lets experiment Grad descent for different values of $\mu$ and $\Sigma$ 

In [8]:

######## GRADIENT DESCENT #############

# d =2
m_data = [1,2]
s_data = [[5, 11], [11, 25]]

####Cholesky gives : [1,2] [3,4]

#Sampling X data
batch_size=100
x_batches = multivariate_normal.rvs(m_data, s_data, (10,batch_size))


#Initial Values
mu_init = [-4, 9]
a,b =  tf.Variable((-2+ 1), dtype="float32"), tf.Variable(-4+2, dtype="float32"),
c,d =  tf.Variable(3+3, dtype="float32"), tf.Variable(1+ 4 ,dtype="float32")
cte_init = 1

mu = tf.Variable(mu_init, dtype="float32")

cte = cte_init
error_mu = [] 
error_sigma = []


error_cte = [] 
mus = []
sigmas = []


ctes = []
max_iters = 700
learning_rate = 0.01
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
 
for itr in range(max_iters): 
    #print(itr)
    for x in x_batches:
        ctes.append(cte)
        Z1 = np.array(random.normal( 0, 1,batch_size)).astype("float32")
        Z2 = np.array(random.normal( 0, 1,batch_size)).astype("float32")
        
        g1= a*Z1 + b*Z2 + mu[0]
        g2 = c*Z1 +  d*Z2 + mu[1]

        A = np.array([[a,b],[c,d]])
        S = np.array(np.matmul(A , np.transpose(A))).astype("float32")
        
        

        g= np.stack([g1,g2],1)
        
       
        
        m0 = mu.numpy()
        
        p_nx = pn( x,m0, S).numpy() # noise (generator) density 
        p_mx = pm0(x, m_data, s_data).numpy() #data density

        
        p_ng = pn( g,m0, S).numpy() # noise (generator) density 
        p_mg = pm0(g, m_data, s_data).numpy() #data density

        grad_cte = 1/cte - p_mx/(cte*p_mx+p_nx) -  p_mg/(cte*p_mg+p_ng)
        grad_cte = np.sum(grad_cte)/batch_size
            
        cte = cte + 0.01*grad_cte
        error_cte.append( (grad_cte) ) 
        if np.isnan(grad_cte):
            break
        opt.minimize(obj_f2, var_list=[ a,b,c,d,mu])
        

Instructions for updating:
`MultivariateNormalFullCovariance` is deprecated, use `MultivariateNormalTriL(loc=loc, scale_tril=tf.linalg.cholesky(covariance_matrix))` instead.


In [16]:
print("Cte Estimate" , cte)
print("True Value" ,1/np.sqrt(np.linalg.det(s_data)*(2*pi)**len(m_data)))

A = [[a.numpy(),b.numpy()],[c.numpy(),d.numpy()]]

print("\n Generator Cov Matrix \n" , np.matmul(A, np.transpose(A)))

print("\n Mu Generator" , mu.numpy())

Cte Estimate 0.07957740371823419
True Value 0.0795774715459475

 Generator Cov Matrix 
 [[ 5.0000005 11.000002 ]
 [11.000002  25.00001  ]]

 Mu Generator [0.9999995 2.0000002]


Indeed The Generator parameters converged to the data distribution parameters

Lets try with other values of $\mu$ $\Sigma$

In [21]:
######## GRADIENT DESCENT #############

# d =2
m_data = [1,2]

s_data = [[25, 39],
       [39, 61]]

# cholesky gives [3,4] [5, 6]
# Sometimes Cholesky is not succesful

#Sampling X data
batch_size=100
x_batches = multivariate_normal.rvs(m_data, s_data, (10,batch_size))


#Initial Values
mu_init = [-4, 9]
a,b =  tf.Variable((-2+ 1), dtype="float32"), tf.Variable(-4+2, dtype="float32"),
c,d =  tf.Variable(3+3, dtype="float32"), tf.Variable(1+ 4 ,dtype="float32")
cte_init = 1

mu = tf.Variable(mu_init, dtype="float32")

cte = cte_init
error_mu = [] 
error_sigma = []


error_cte = [] 
mus = []
sigmas = []


ctes = []
max_iters = 700
learning_rate = 0.01
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
 
for itr in range(max_iters): 
    #print(itr)
    for x in x_batches:
        ctes.append(cte)
        Z1 = np.array(random.normal( 0, 1,batch_size)).astype("float32")
        Z2 = np.array(random.normal( 0, 1,batch_size)).astype("float32")
        
        g1= a*Z1 + b*Z2 + mu[0]
        g2 = c*Z1 +  d*Z2 + mu[1]

        A = np.array([[a,b],[c,d]])
        S = np.array(np.matmul(A , np.transpose(A))).astype("float32")
        
        

        g= np.stack([g1,g2],1)
        
       
        
        m0 = mu.numpy()
        
        p_nx = pn( x,m0, S).numpy() # noise (generator) density 
        p_mx = pm0(x, m_data, s_data).numpy() #data density

        
        p_ng = pn( g,m0, S).numpy() # noise (generator) density 
        p_mg = pm0(g, m_data, s_data).numpy() #data density

        grad_cte = 1/cte - p_mx/(cte*p_mx+p_nx) -  p_mg/(cte*p_mg+p_ng)
        grad_cte = np.sum(grad_cte)/batch_size
            
        cte = cte + 0.01*grad_cte
        error_cte.append( (grad_cte) ) 
        if np.isnan(grad_cte):
            break
        opt.minimize(obj_f2, var_list=[ a,b,c,d,mu])
        

In [22]:
print("Cte Estimate" , cte)
print("True Value" ,1/np.sqrt(np.linalg.det(s_data)*(2*pi)**len(m_data)))

A = [[a.numpy(),b.numpy()],[c.numpy(),d.numpy()]]

print("\n Generator Cov Matrix \n" , np.matmul(A, np.transpose(A)))

print("\n Mu Generator" , mu.numpy())

Cte Estimate 0.079576065646111
True Value 0.07957747154594781

 Generator Cov Matrix 
 [[24.999825 38.99974 ]
 [38.99974  60.999622]]

 Mu Generator [1.0030813 2.00482  ]
